In [1]:
import cv2
from pathlib import Path
from random import *
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.feature import hog
from imutils import face_utils
import dlib
import os
import pickle
np.random.seed(1000)


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
frames = []
labels = []
for file in os.listdir('output_confusion/'):
    if file[-10:] == 'frames.pkl':
        with open('output_confusion/'+file, 'rb') as f:
            frames.append(pickle.load(f))
    elif file[-10:] == 'labels.pkl':
        with open('output_confusion/'+file, 'rb') as f:
            labels.append(pickle.load(f))

In [4]:
print(len(frames), len(labels))

9067 9067


In [5]:
from sklearn.model_selection import train_test_split
train_clips, test_clips, train_clips_labels, test_clips_labels = \
    train_test_split(frames, labels, test_size=0.2, random_state=42)

In [6]:
train_images, test_images, train_labels, test_labels = [], [], [], []

for clip, label in zip(train_clips, train_clips_labels):
    try:
        train_images, train_labels = train_images + clip, train_labels + [label[0]] * len(clip)
    except:
        continue

for clip, label in zip(test_clips, test_clips_labels):
    try:
        test_images, test_labels = test_images + clip, test_labels + [label[0]] * len(clip)
    except:
        continue
    
print(len(train_images), len(train_labels), len(test_images), len(test_labels))

1944475 1944475 482212 482212


In [7]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
for i in range(len(train_images)):
    train_images[i] = clahe.apply(train_images[i])
for i in range(len(test_images)):
    test_images[i] = clahe.apply(test_images[i])

In [8]:
train_images, test_images, train_labels, test_labels = np.asarray(train_images), np.asarray(test_images), np.asarray(train_labels), np.asarray(test_labels)

In [9]:
test_images = np.expand_dims(test_images, axis=3)
train_images = np.expand_dims(train_images, axis=3)

In [10]:
train_labels //= 2
test_labels //= 2

In [17]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=288, input_shape=(48,48,1), kernel_size=3, padding='valid', activation='elu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=288, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.Conv2D(filters=288, kernel_size=3, strides=2, padding='valid', activation='elu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(filters=144, kernel_size=3, padding='valid', activation='elu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=144, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.Conv2D(filters=144, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.MaxPooling2D(pool_size=3, strides=2),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(filters=48, kernel_size=3, padding='valid', activation='elu'),   
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=48, kernel_size=3, padding='valid', activation='elu'),   
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=48, kernel_size=3, padding='valid', activation='elu'),   
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=4, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(2, activation='softmax')
])

In [18]:
model.compile(optimizer='adamax',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
weights = {0:0.55091863, 1:5.40979479}
print(weights)

{0: 0.55091863, 1: 5.40979479}


In [20]:
model.fit(train_images, train_labels, epochs=5, class_weight=weights, batch_size=50)

  ...
    to  
  ['...']
Train on 1944475 samples
Epoch 1/5
1944475/1944475 [==============================] - 5266s 3ms/sample - loss: 0.4282 - accuracy: 0.7810
Epoch 2/5
1944475/1944475 [==============================] - 4691s 2ms/sample - loss: 0.2874 - accuracy: 0.8657
Epoch 3/5
1944475/1944475 [==============================] - 2095s 1ms/sample - loss: 0.2384 - accuracy: 0.8914
Epoch 4/5
1944475/1944475 [==============================] - 2157s 1ms/sample - loss: 0.2102 - accuracy: 0.9060
Epoch 5/5
1944475/1944475 [==============================] - 2092s 1ms/sample - loss: 0.1911 - accuracy: 0.9152


In [21]:
test_loss, test_acc = model.evaluate(test_images,  test_labels)

482212/482212 [==============================] - 160s 331us/sample - loss: 0.6125 - accuracy: 0.8208


In [22]:
model.save('confusion.h5')